In [ ]:
CREATE TABLE TempleDonations (
    DonationID NUMBER AUTOINCREMENT START 1 INCREMENT 1 PRIMARY KEY,
    DonorName VARCHAR(100) NOT NULL,
    DonorEmail VARCHAR(100),
    DonorPhone VARCHAR(20),
    DonationAmount NUMBER(10, 2) NOT NULL,
    DonationDate DATE NOT NULL,
    DonationMethod STRING, -- e.g., Cash, Credit Card, Online Transfer, etc.
    Purpose STRING, -- e.g., General Fund, Special Event, etc.
    Remarks STRING
);


In [ ]:
import streamlit as st
from datetime import datetime
from snowflake.snowpark.context import get_active_session
import pandas as pd
import time

# Initialize Snowpark session
session = get_active_session()

# Function to insert data into the TempleDonations table
def insert_into_temple_donations( donor_name, donor_email, donor_phone, donation_amount, donation_date, donation_method, purpose, remarks):
    """
    This function inserts data into the TempleDonations table using Snowflake SQL.
    """
    query = f"""
    INSERT INTO TempleDonations ( DonorName, DonorEmail, DonorPhone, DonationAmount, DonationDate, DonationMethod, Purpose, Remarks)
    VALUES ( '{donor_name}', '{donor_email}', '{donor_phone}', {donation_amount}, '{donation_date}', '{donation_method}', '{purpose}', '{remarks}');
    """
    return query

# Streamlit app title
st.title("Temple Donations")

# Form to add a new donation
with st.form(key='donation_form'):
    donor_name = st.text_input("Donor Name", max_chars=100)
    donor_email = st.text_input("Donor Email", max_chars=100)
    donor_phone = st.text_input("Donor Phone", max_chars=20)
    donation_amount = st.number_input("Donation Amount", min_value=0.0)
    donation_date = st.date_input("Donation Date", datetime.today())
    donation_method = st.selectbox("Donation Method", ["Cash", "Credit Card", "Online Transfer", "Other"])
    purpose = st.text_input("Purpose", max_chars=100)
    remarks = st.text_area("Remarks")
    
    # Simulate a unique DonationID (in practice, this could be managed differently)
    #donation_id = st.number_input("Donation ID", min_value=1, value=1, step=1)
    
    #submit_button = st.form_submit_button(label='Submit')

# If the form is submitted, add the donation
if st.button("SUBMIT"):
    result = insert_into_temple_donations( donor_name, donor_email, donor_phone, donation_amount, donation_date, donation_method, purpose, remarks)
    session.sql(result).collect()
    st.success("Donation added successfully!")

# Display donation summary
st.header("Donation Summary")

# Query to fetch data from the dynamic table
query = """
SELECT total_donations, donor_count
FROM donation_summary_dynamic
"""

# Fetch the data
df = session.sql(query)

# Display the data
st.write("Total Donations and Donor Count")
st.write("Refreshing in 60 seconds...")
st.write(df)
time.sleep(70)
st.experimental_rerun()



In [ ]:
CREATE OR REPLACE DYNAMIC TABLE donation_summary_dynamic
  WAREHOUSE = 'COMPUTE_WH'
  LAG= '1 minute'
  AS
SELECT
  SUM(DonationAmount) AS total_donations,
  COUNT(*) AS donor_count
FROM
  TempleDonations;


In [ ]:
select * from TempleDonations;